In [166]:
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import plotly.express as px
import geopandas as gpd
import contextily as ctx

# Definir los nombres de las columnas
Variables= ['cod_reg', 'cod_dep', 'cod_loc', 'pad', 'block', 'EP', 'uni', 'sup_predio','sup_edificada', 'V_cat_terreno',
            'V_cat_mejoras',  'V_cat_total', 'V_impuestos', 'Fecha_UDJ', 'Vigencia_UDJ']  

# Especificar los tipos de datos para las columnas
dtypes = {
    'cod_reg': str,
    'cod_dep': str,
    'cod_loc': str,  # O str, dependiendo de tus datos
    'pad': int,
    'block': str,
    'EP': str,
    'uni': int,
    'sup_predio': int,
    'sup_edificada': int,
    'V_cat_terreno': int,
    'V_cat_mejoras': int,
    'V_cat_total': int,
    'V_impuestos': int,
    'Fecha_UDJ': bool,
    'Vigencia_UDJ': bool
}

P_Urbanos = pd.read_csv('C:/Users/vfernand/Desktop/archivos proyecto PYTHON/DatosAbiertosDNC(2024-09)/Padrones Urbanos.csv',
sep=',', encoding='cp1252', header=None, names=Variables, dtype={4: str, 5: str}  )

# Mostrar las primeras filas del archivo
P_Urbanos.head(8)



,cod_reg,cod_dep,cod_loc,pad,block,EP,uni,sup_predio,sup_edificada,V_cat_terreno,V_cat_mejoras,V_cat_total,V_impuestos,Fecha_UDJ,Vigencia_UDJ
0,CO,A,AA,1,NaN,NaN,0,1473,438,396202,1644448,2040650,2040650,/ /,/ /
1,CO,A,AA,2,NaN,NaN,0,246,112,149880,640149,790029,790029,/ /,/ /
2,CO,A,AA,3,NaN,NaN,0,123,99,70630,489035,559665,559665,/ /,/ /
3,CO,A,AA,4,NaN,NaN,0,234,54,132215,220728,352943,352943,/ /,/ /
4,CO,A,AA,6,NaN,NaN,0,505,75,274276,431225,705501,705501,/ /,/ /
5,CO,A,AA,10,NaN,NaN,0,462,283,303908,1339819,1643727,1643727,/ /,/ /
6,CO,A,AA,11,NaN,NaN,0,440,139,290376,584098,874474,874474,/ /,/ /
7,CO,A,AA,12,NaN,NaN,0,470,152,308792,454518,763310,763310,/ /,/ /


In [167]:
# Filtrar filas donde 'cod_dep' es igual a 'V' que es Montevideo
filtro_dep = P_Urbanos[P_Urbanos['cod_dep'] == 'V']


# Mostrar el DataFrame filtrado
filtro_dep.head()

,cod_reg,cod_dep,cod_loc,pad,block,EP,uni,sup_predio,sup_edificada,V_cat_terreno,V_cat_mejoras,V_cat_total,V_impuestos,Fecha_UDJ,Vigencia_UDJ
812667,CO,V,AA,5,NaN,NaN,0,130,138,669500,374348,1043848,1043848,/ /,/ /
812668,CO,V,AA,7,NaN,NaN,0,206,168,873257,319436,1192693,1192693,/ /,/ /
812669,CO,V,AA,8,NaN,NaN,0,175,341,596274,3670770,4267044,4267044,28/11/2018,28/11/2023
812670,CO,V,AA,9,NaN,NaN,0,171,143,583071,543110,1126181,1126181,/ /,/ /
812671,CO,V,AA,14,NaN,NaN,0,196,303,893558,325618,1219176,1219176,/ /,/ /


In [168]:
# Eliminar los apartamentos duplicados y quedarte solo con un registro por padrón
filtro_dep = filtro_dep.drop_duplicates(subset='pad').reset_index(drop=True)

# Mostrar el DataFrame filtrado
filtro_dep.head()

,cod_reg,cod_dep,cod_loc,pad,block,EP,uni,sup_predio,sup_edificada,V_cat_terreno,V_cat_mejoras,V_cat_total,V_impuestos,Fecha_UDJ,Vigencia_UDJ
0,CO,V,AA,5,NaN,NaN,0,130,138,669500,374348,1043848,1043848,/ /,/ /
1,CO,V,AA,7,NaN,NaN,0,206,168,873257,319436,1192693,1192693,/ /,/ /
2,CO,V,AA,8,NaN,NaN,0,175,341,596274,3670770,4267044,4267044,28/11/2018,28/11/2023
3,CO,V,AA,9,NaN,NaN,0,171,143,583071,543110,1126181,1126181,/ /,/ /
4,CO,V,AA,14,NaN,NaN,0,196,303,893558,325618,1219176,1219176,/ /,/ /


In [169]:
#CUantas filas tiene el archivo
num_filas = filtro_dep.shape[0]

print(num_filas)

200177


In [170]:
# Reemplazar cadenas vacías o '/  /' por NaN
filtro_dep['Fecha_UDJ'] = filtro_dep['Fecha_UDJ'].replace(['', '/  /'], pd.NA)

# Separar la columna 'Fecha' en tres nuevas columnas, manejando valores NaN
filtro_dep[['dia', 'mes', 'anio']] = filtro_dep['Fecha_UDJ'].str.split('/', expand=True)

# Convertir las nuevas columnas a tipo entero, usando fillna para evitar errores
filtro_dep['dia'] = filtro_dep['dia'].astype('Int64')  # 'Int64' permite valores NA
filtro_dep['mes'] = filtro_dep['mes'].astype('Int64')
filtro_dep['anio'] = filtro_dep['anio'].astype('Int64')

# Mostrar el DataFrame resultante
filtro_dep.head()

,cod_reg,cod_dep,cod_loc,pad,block,EP,uni,sup_predio,sup_edificada,V_cat_terreno,V_cat_mejoras,V_cat_total,V_impuestos,Fecha_UDJ,Vigencia_UDJ,dia,mes,anio
0,CO,V,AA,5,NaN,NaN,0,130,138,669500,374348,1043848,1043848,<NA>,/ /,<NA>,<NA>,<NA>
1,CO,V,AA,7,NaN,NaN,0,206,168,873257,319436,1192693,1192693,<NA>,/ /,<NA>,<NA>,<NA>
2,CO,V,AA,8,NaN,NaN,0,175,341,596274,3670770,4267044,4267044,28/11/2018,28/11/2023,28,11,2018
3,CO,V,AA,9,NaN,NaN,0,171,143,583071,543110,1126181,1126181,<NA>,/ /,<NA>,<NA>,<NA>
4,CO,V,AA,14,NaN,NaN,0,196,303,893558,325618,1219176,1219176,<NA>,/ /,<NA>,<NA>,<NA>


In [171]:
# Ver los tipos de datos de cada columna
tipos_datos = filtro_dep.dtypes

# Mostrar los tipos de datos
print(tipos_datos)

cod_reg          object
cod_dep          object
cod_loc          object
pad               int64
block            object
EP               object
uni               int64
sup_predio        int64
sup_edificada     int64
V_cat_terreno     int64
V_cat_mejoras     int64
V_cat_total       int64
V_impuestos       int64
Fecha_UDJ        object
Vigencia_UDJ     object
dia               Int64
mes               Int64
anio              Int64
dtype: object


In [172]:
filtro_dep = filtro_dep[filtro_dep['anio'] >= 1997]

# Contar la cantidad de registros por año
conteo_por_anio = filtro_dep.groupby('anio').size().reset_index(name='cantidad_registros')

print(conteo_por_anio)

    anio  cantidad_registros
0   1997                   1
1   2001                   5
2   2002                 279
3   2003                 862
4   2004                1680
5   2005                1416
6   2006                1457
7   2007                1780
8   2008                2319
9   2009                2081
10  2010                2644
11  2011                3130
12  2012                2484
13  2013                2786
14  2014                2606
15  2015                2578
16  2016                2615
17  2017                3189
18  2018                2834
19  2019                2915
20  2020                2695
21  2021                3260
22  2022                6510
23  2023                6699
24  2024                4456


In [173]:
#Permisos Intendencia Montevideo
# Leer el archivo CSV y asignar los nombres a las columnas
Permisos = pd.read_csv('C:/Users/vfernand/Desktop/archivos proyecto PYTHON/permisos_construccion.csv', sep=';')

Permisos.rename(columns={'padron': 'pad'}, inplace=True)
# Mostrar las primeras filas del archivo
Permisos.head()

,pad,destino,area,calle,puerta,letra,anio,mes,expediente,tipo_obra
0,2,Vivienda ...,26.00,MIGUELETE ...,1420.0,,2000,9,4113-016778-00,Obra Nueva
1,8,Comercio ...,315.00,LA PAZ ...,1439.0,,2013,7,4113-007575-13,Obra Nueva
2,8,Comercio ...,5.00,LA PAZ ...,1439.0,,2019,3,2018-4113-98-001306,Regularizacion - Año
3,9,Vivienda ...,14.84,LA PAZ ...,1431.0,,1998,12,4113-016126-98,Reforma
4,15,Vivienda ...,49.85,PIEDRA ALTA ...,1779.0,,2004,3,4113-014456-00,Modificacion en Obra


In [174]:
# Ver los tipos de datos de cada columna
tipos_datos = Permisos.dtypes

# Mostrar los tipos de datos
print(tipos_datos)

pad             int64
destino        object
area          float64
calle          object
puerta        float64
letra          object
anio            int64
mes             int64
expediente     object
tipo_obra      object
dtype: object


In [175]:
#Crear en permisos la variable cod_dep
Permisos['cod_dep'] = 'V'
Permisos.head()

,pad,destino,area,calle,puerta,letra,anio,mes,expediente,tipo_obra,cod_dep
0,2,Vivienda ...,26.00,MIGUELETE ...,1420.0,,2000,9,4113-016778-00,Obra Nueva,V
1,8,Comercio ...,315.00,LA PAZ ...,1439.0,,2013,7,4113-007575-13,Obra Nueva,V
2,8,Comercio ...,5.00,LA PAZ ...,1439.0,,2019,3,2018-4113-98-001306,Regularizacion - Año,V
3,9,Vivienda ...,14.84,LA PAZ ...,1431.0,,1998,12,4113-016126-98,Reforma,V
4,15,Vivienda ...,49.85,PIEDRA ALTA ...,1779.0,,2004,3,4113-014456-00,Modificacion en Obra,V


In [176]:
# Calcular la suma total de 'conteo_destino'
conteo_pad = Permisos['pad'].count()

# Mostrar la suma total
print(f"Suma total de conteo_pad: {conteo_pad}")

Suma total de conteo_pad: 39428


In [177]:
### Saber cantidad de filas y columnas
Permisos.shape

(39428, 11)

In [178]:
# Ordenar el DataFrame por Padron (ascendente), anio, mes, area en orden descendente me quedo con el ùltimo permiso aprobado para ese padron y el area màs grande de esa fecha.
Permisos_ordenado = Permisos.sort_values(by=['pad','anio', 'mes', 'area'], ascending=[True,False, False, False])

# Mostrar el DataFrame ordenado
Permisos_ordenado.head(15)


,pad,destino,area,calle,puerta,letra,anio,mes,expediente,tipo_obra,cod_dep
0,2,Vivienda ...,26.00,MIGUELETE ...,1420.0,,2000,9,4113-016778-00,Obra Nueva,V
2,8,Comercio ...,5.00,LA PAZ ...,1439.0,,2019,3,2018-4113-98-001306,Regularizacion - Año,V
1,8,Comercio ...,315.00,LA PAZ ...,1439.0,,2013,7,4113-007575-13,Obra Nueva,V
3,9,Vivienda ...,14.84,LA PAZ ...,1431.0,,1998,12,4113-016126-98,Reforma,V
4,15,Vivienda ...,49.85,PIEDRA ALTA ...,1779.0,,2004,3,4113-014456-00,Modificacion en Obra,V
6,27,Comercio ...,284.00,LA PAZ ...,1461.0,,2021,9,2021-4113-98-000660,Reforma a Regularizar,V
5,27,Comercio ...,320.00,LA PAZ ...,1461.0,,2014,9,2016-4118-98-000212,Obra Nueva,V
8,32,Comercio ...,353.00,PIEDRA ALTA ...,1771.0,,2008,4,2016-4113-98-001655,Regularizacion - Año,V
7,32,Comercio ...,36.00,PIEDRA ALTA ...,1771.0,,2008,4,2016-4113-98-001655,Obra Nueva,V
9,43,"Vivienda,Comercio,Otros ...",147.00,MINAS ...,1803.0,,2022,11,2022-4113-98-001048,Incorporacion A.P.H.,V


In [179]:
# Primero, eliminamos duplicados por padròn para quedarnos con el ùtimo registro en año para ese padròn.
p_sin_duplicados = Permisos_ordenado.drop_duplicates(subset=['pad','anio','mes'])

p_sin_duplicados.head(15)

,pad,destino,area,calle,puerta,letra,anio,mes,expediente,tipo_obra,cod_dep
0,2,Vivienda ...,26.00,MIGUELETE ...,1420.0,,2000,9,4113-016778-00,Obra Nueva,V
2,8,Comercio ...,5.00,LA PAZ ...,1439.0,,2019,3,2018-4113-98-001306,Regularizacion - Año,V
1,8,Comercio ...,315.00,LA PAZ ...,1439.0,,2013,7,4113-007575-13,Obra Nueva,V
3,9,Vivienda ...,14.84,LA PAZ ...,1431.0,,1998,12,4113-016126-98,Reforma,V
4,15,Vivienda ...,49.85,PIEDRA ALTA ...,1779.0,,2004,3,4113-014456-00,Modificacion en Obra,V
6,27,Comercio ...,284.00,LA PAZ ...,1461.0,,2021,9,2021-4113-98-000660,Reforma a Regularizar,V
5,27,Comercio ...,320.00,LA PAZ ...,1461.0,,2014,9,2016-4118-98-000212,Obra Nueva,V
8,32,Comercio ...,353.00,PIEDRA ALTA ...,1771.0,,2008,4,2016-4113-98-001655,Regularizacion - Año,V
9,43,"Vivienda,Comercio,Otros ...",147.00,MINAS ...,1803.0,,2022,11,2022-4113-98-001048,Incorporacion A.P.H.,V
12,52,Vivienda ...,50.00,LA PAZ ...,1577.0,,2005,2,4113-000339-05,Reforma,V


In [180]:
#Eliminamos fechas anteriores a 1999
p_sin_duplicados = p_sin_duplicados[p_sin_duplicados['anio'] >= 1997]

# Contar cuántos registros hay por 'anio' y 'mes'
resultado = p_sin_duplicados.groupby(['anio']).size().reset_index(name ='Cantidad_de_registros')

# Mostrar el resultado
print(resultado)

    anio  Cantidad_de_registros
0   1997                     19
1   1998                   1044
2   1999                   1237
3   2000                   1088
4   2001                    655
5   2002                    688
6   2003                    986
7   2004                   1115
8   2005                   1358
9   2006                   1224
10  2007                   1309
11  2008                   1430
12  2009                   1340
13  2010                   1330
14  2011                   1249
15  2012                   1156
16  2013                   1071
17  2014                   1169
18  2015                   1003
19  2016                    662
20  2017                    963
21  2018                    794
22  2019                    798
23  2020                    559
24  2021                    706
25  2022                    784
26  2023                    626
27  2024                    269


In [181]:
print(resultado.columns)


Index(['anio', 'Cantidad_de_registros'], dtype='object')


In [182]:
# Machear los DataFrames usando un left join
cat_per = pd.merge(filtro_dep, Permisos, on=['anio','mes','pad'], how='left')

# Machear los DataFrames usando un left join
per_cat = pd.merge(p_sin_duplicados,filtro_dep, on=['anio','mes','pad'], how='left')

In [183]:
#Mostrar cantidad de filas y columnas del primer Dataframe resultante
cat_per.shape

(63986, 26)

In [184]:
#Mosstrar el segundo Dataframe resultante
per_cat.shape

(26632, 26)

In [185]:
# Guardar el DataFrame macheado como CSV de catastro
cat_per.to_csv('C:/Users/vfernand/Desktop/archivos proyecto PYTHON/cat_per.csv', index=False)

In [186]:
# Guardar el DataFrame macheado como CSV de permisos
per_cat.to_csv('C:/Users/vfernand/Desktop/archivos proyecto PYTHON/per_cat.csv', index=False)

In [187]:
#Caluclar la proporcion y cantidad de registros por regimen
conteo_codreg = per_cat.groupby('cod_reg').size().reset_index(name='cantidad_expedientes')
conteo_codreg['porcentaje'] = ((conteo_codreg['cantidad_expedientes'] / conteo_codreg['cantidad_expedientes'].sum()) * 100).round(2)
conteo_codreg

,cod_reg,cantidad_expedientes,porcentaje
0,CO,1071,93.29
1,PH,77,6.71


In [188]:
#Caluclar la proporcion y cantidad de registros por destino
conteo_destino = per_cat.groupby('destino').size().reset_index(name='cantidades por destino')
conteo_destino['porcentaje_destino'] = ((conteo_destino['cantidades por destino'] / conteo_destino['cantidades por destino'].sum()) * 100).round(2)
conteo_destino

,destino,cantidades por destino,porcentaje_destino
0,Comercio ...,4567,17.15
1,"Comercio, Industria, Otros ...",3,0.01
2,"Comercio,Industria ...",22,0.08
3,"Comercio,Otros ...",102,0.38
4,Desconocido ...,1426,5.35
5,Industria ...,828,3.11
6,"Industria,Otros ...",26,0.10
7,Otros ...,835,3.14
8,Vivienda ...,16687,62.66
9,"Vivienda, Industria, Otros ...",5,0.02


In [189]:
#Ver todos los destinos
print(per_cat['destino'].unique())

['Vivienda                                          '
 'Comercio                                          '
 'Vivienda,Comercio,Otros                           '
 'Vivienda,Otros                                    '
 'Otros                                             '
 'Industria                                         '
 'Desconocido                                       '
 'Comercio,Industria                                '
 'Vivienda,Comercio                                 '
 'Comercio,Otros                                    '
 'Vivienda,Industria                                '
 'Industria,Otros                                   '
 'Vivienda, Industria, Otros                        '
 'Comercio, Industria, Otros                        '
 'Vivienda,Comercio,Industria                       ']


In [190]:
per_cat['destino'] = per_cat['destino'].str.strip()

In [191]:
#Recodificar destinos
mapeo_destinos = {
    'Comercio': 'Comercio',
    'Comercio, Industria, Otros': 'Varios destinos',
    'Comercio,Industria': 'Varios destinos',
    'Comercio,Otros': 'Varios destinos',
    'Industria': 'Industria',
    'Industria,Otros': 'Varios destinos',
    'Vivienda': 'Vivienda',
    'Vivienda, Industria, Otros': 'Varios destinos',
    'Vivienda,Comercio': 'Varios destinos',
    'Vivienda,Comercio,Industria': 'Varios destinos',
    'Vivienda,Comercio,Otros': 'Varios destinos',
    'Vivienda,Industria': 'Varios destinos',
    'Vivienda,Otros': 'Varios destinos',
    'Desconocido' : 'Sin destino definido',
    'Otros': 'Sin destino definido'         
}

per_cat['Nuevo_Destino'] = per_cat['destino'].map(mapeo_destinos)

per_cat.head()

,pad,destino,area,calle,puerta,letra,anio,mes,expediente,tipo_obra,...,sup_predio,sup_edificada,V_cat_terreno,V_cat_mejoras,V_cat_total,V_impuestos,Fecha_UDJ,Vigencia_UDJ,dia,Nuevo_Destino
0,2,Vivienda,26.00,MIGUELETE ...,1420.0,,2000,9,4113-016778-00,Obra Nueva,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,Vivienda
1,8,Comercio,5.00,LA PAZ ...,1439.0,,2019,3,2018-4113-98-001306,Regularizacion - Año,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,Comercio
2,8,Comercio,315.00,LA PAZ ...,1439.0,,2013,7,4113-007575-13,Obra Nueva,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,Comercio
3,9,Vivienda,14.84,LA PAZ ...,1431.0,,1998,12,4113-016126-98,Reforma,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,Vivienda
4,15,Vivienda,49.85,PIEDRA ALTA ...,1779.0,,2004,3,4113-014456-00,Modificacion en Obra,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,Vivienda


In [192]:
#ver los nuevos deestinos
print(per_cat['Nuevo_Destino'].unique())

# Agrupar por la nueva categoría
conteo_Nuevo_Destino = per_cat.groupby('Nuevo_Destino').size().reset_index(name='cantidades por destino')
conteo_Nuevo_Destino

['Vivienda' 'Comercio' 'Varios destinos' 'Sin destino definido'
 'Industria']


,Nuevo_Destino,cantidades por destino
0,Comercio,4567
1,Industria,828
2,Sin destino definido,2261
3,Varios destinos,2289
4,Vivienda,16687


In [193]:
conteo_Nuevo_Destino['porcentaje_Nuevo_Destino'] = ((conteo_Nuevo_Destino['cantidades por destino'] / conteo_Nuevo_Destino['cantidades por destino'].sum()) * 100).round(2)
conteo_Nuevo_Destino

,Nuevo_Destino,cantidades por destino,porcentaje_Nuevo_Destino
0,Comercio,4567,17.15
1,Industria,828,3.11
2,Sin destino definido,2261,8.49
3,Varios destinos,2289,8.59
4,Vivienda,16687,62.66


In [194]:
#Ver destinos y sus cantidades
conteo_tipo_obra = per_cat.groupby('tipo_obra').size().reset_index(name='cantidad por tipo de obra')
conteo_tipo_obra

,tipo_obra,cantidad por tipo de obra
0,A regularizar,20
1,Autorizada,66
2,Cielo A. Autorizada,2
3,Demolicion,60
4,Desconocido,1109
5,Estacionamiento,42
6,Galpon,49
7,Incorporacion A.P.H.,748
8,Incorporación a PH,13
9,Marquesina,16


In [195]:
tabla_Regimen_tipoObra = pd.pivot_table(per_cat, index='tipo_obra', columns='cod_reg', aggfunc='size', fill_value=0)
tabla_Regimen_tipoObra

cod_reg,CO,PH
tipo_obra,,
Demolicion,1,0
Desconocido,1,0
Incorporacion A.P.H.,3,1
Incorporación a PH,1,0
Modificacion en Obra,0,1
Obra Nueva,453,17
Obra nueva / ampliación,1,0
Reciclaje,0,1
Reforma,281,13


In [196]:
#Levantar nuevo archivo descargado de la intendencia con catastro incorporado
Cat_Permisos = pd.read_csv('C:/Users/vfernand/Desktop/archivos proyecto PYTHON/v_ce_permisos_construccion_geom.csv', sep=';' , encoding='cp1252')

Cat_Permisos = Cat_Permisos[Cat_Permisos['ANIO_APRO'] >= 1997]


In [197]:
Cat_Permisos.head(3)

,GID,EXPEDIENTE,FECHA_INI,FECHA_APRO,MES_APRO,ANIO_APRO,PADRON,UNIDAD,DSC_TIPO_T,DSC_DESTIN,DSC_REGIME,DSC_TIPO_O,DSC_CATEGO,CONSTRUCTOR,AREA_EDIF
0,25988748,4113-016778-00,10/08/2000,06/09/2000,9,2000,2,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Propiedad Horizontal,Obra Nueva,Económico o Sencillo,Desconocido,"26,00"
1,25988984,2018-4113-98-001306,19/12/2018,06/03/2019,3,2019,8,NaN,OBRAS/CONSTRUCCIONES,Comercio,Común,Regularizacion - Año,Mediano,Desconocido,"5,00"
2,25988984,4113-007575-13,03/07/2013,03/07/2013,7,2013,8,NaN,OBRAS/CONSTRUCCIONES,Comercio,Común,Obra Nueva,Mediano,Desconocido,"315,00"


In [198]:
# Definir fechas
Cat_Permisos['FECHA_INI'] = pd.to_datetime(Cat_Permisos['FECHA_INI'], format='%d/%m/%Y', errors='coerce')
Cat_Permisos['FECHA_APRO'] = pd.to_datetime(Cat_Permisos['FECHA_APRO'], format='%d/%m/%Y', errors='coerce')

In [199]:
print(Cat_Permisos['AREA_EDIF'].dtype)

# Reemplazar las comas y convertir a float
Cat_Permisos['AREA_EDIF'] = Cat_Permisos['AREA_EDIF'].fillna(0).astype(str).str.replace(',', '').astype(float)

# Convertir la columna a enteros
Cat_Permisos['AREA_EDIF'] = Cat_Permisos['AREA_EDIF'].astype('Int64')

# Verificar los tipos de datos
tipos_datos = Cat_Permisos.dtypes

print(tipos_datos)

object
GID                     int64
EXPEDIENTE             object
FECHA_INI      datetime64[ns]
FECHA_APRO     datetime64[ns]
MES_APRO                int64
ANIO_APRO               int64
PADRON                  int64
UNIDAD                 object
DSC_TIPO_T             object
DSC_DESTIN             object
DSC_REGIME             object
DSC_TIPO_O             object
DSC_CATEGO             object
CONSTRUCTOR            object
AREA_EDIF               Int64
dtype: object


In [200]:
#Definir fechas
Cat_Permisos['FECHA_INI'] = pd.to_datetime(Cat_Permisos['FECHA_INI'])
Cat_Permisos['FECHA_APRO'] = pd.to_datetime(Cat_Permisos['FECHA_APRO'])

In [201]:
#Fecha de aprobado el permiso y fecha de inicio del mismo
Cat_Permisos['T_Demora'] = (Cat_Permisos['FECHA_APRO'] - Cat_Permisos['FECHA_INI']).dt.days
Cat_Permisos['año_ini'] = Cat_Permisos['FECHA_INI'].dt.year
Cat_Permisos.shape

Cat_Permisos

,GID,EXPEDIENTE,FECHA_INI,FECHA_APRO,MES_APRO,ANIO_APRO,PADRON,UNIDAD,DSC_TIPO_T,DSC_DESTIN,DSC_REGIME,DSC_TIPO_O,DSC_CATEGO,CONSTRUCTOR,AREA_EDIF,T_Demora,año_ini
0,25988748,4113-016778-00,2000-08-10,2000-09-06,9,2000,2,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Propiedad Horizontal,Obra Nueva,Económico o Sencillo,Desconocido,2600,27,2000
1,25988984,2018-4113-98-001306,2018-12-19,2019-03-06,3,2019,8,NaN,OBRAS/CONSTRUCCIONES,Comercio,Común,Regularizacion - Año,Mediano,Desconocido,500,77,2018
2,25988984,4113-007575-13,2013-07-03,2013-07-03,7,2013,8,NaN,OBRAS/CONSTRUCCIONES,Comercio,Común,Obra Nueva,Mediano,Desconocido,31500,0,2013
3,25989011,4113-016126-98,1998-08-24,1998-12-01,12,1998,9,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Común,Reforma,Mediano,Desconocido,1484,99,1998
4,25987893,4113-014456-00,2000-06-30,2004-03-26,3,2004,15,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Propiedad Horizontal,Modificacion en Obra,Confortable,Desconocido,4985,1365,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40781,90215637,GERPC-7-779506,2023-12-08,2024-05-20,5,2024,432190,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Reforma,Desconocido,Desconocido,15100,164,2023
40782,90215624,GERPC-7-792529,2023-12-28,2024-09-13,9,2024,432205,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,327365,260,2023
40783,90215646,GERPC-7-792051,2023-12-27,2024-05-08,5,2024,432207,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,117230,133,2023
40784,93662054,GERPC-7-843816,2024-03-14,2024-08-02,8,2024,432234,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,862714,141,2024


In [202]:
PROM_DEMORA_por_año = Cat_Permisos.groupby('año_ini')['T_Demora'].mean().reset_index()

PROM_DEMORA_por_año.rename(columns={'T_Demora': 'Promedio_T_Demora'}, inplace=True)

PROM_DEMORA_por_año['Promedio_T_Demora'] = PROM_DEMORA_por_año['Promedio_T_Demora'].astype(int)

PROM_DEMORA_por_año

,año_ini,Promedio_T_Demora
0,1997,714
1,1998,732
2,1999,665
3,2000,700
4,2001,721
5,2002,229
6,2003,37
7,2004,29
8,2005,25
9,2006,28


In [203]:
#VER CONTENIDO DE LAS VARIABLES
print(Cat_Permisos['DSC_CATEGO'].unique())
print(Cat_Permisos['DSC_DESTIN'].unique())

['Económico o Sencillo' 'Mediano' 'Confortable' 'Desconocido' 'SUNTUOSO']
['Vivienda' 'Comercio' 'Vivienda,Comercio,Otros' 'Vivienda,Otros' 'Otros'
 'Desconocido' 'Industria' 'Comercio,Industria' 'Vivienda,Comercio'
 'Comercio,Otros' 'Vivienda,Industria' 'Industria,Otros'
 'Vivienda, Industria, Otros' 'Comercio, Industria, Otros'
 'Vivienda,Comercio,Industria']


In [204]:
#Recodificar destinos
mapeo_DSC_DESTIN = {
    'Comercio': 'Comercio',
    'Comercio, Industria, Otros': 'Varios destinos',
    'Comercio,Industria': 'Varios destinos',
    'Comercio,Otros': 'Varios destinos',
    'Industria': 'Industria',
    'Industria,Otros': 'Varios destinos',
    'Vivienda': 'Vivienda',
    'Vivienda, Industria, Otros': 'Varios destinos',
    'Vivienda,Comercio': 'Varios destinos',
    'Vivienda,Comercio,Industria': 'Varios destinos',
    'Vivienda,Comercio,Otros': 'Varios destinos',
    'Vivienda,Industria': 'Varios destinos',
    'Vivienda,Otros': 'Varios destinos',
    'Desconocido' : 'Sin destino definido',
    'Otros': 'Sin destino definido'         
}

Cat_Permisos['Nuevo_Destino'] = Cat_Permisos['DSC_DESTIN'].map(mapeo_DSC_DESTIN)

print(Cat_Permisos['Nuevo_Destino'].unique())

['Vivienda' 'Comercio' 'Varios destinos' 'Sin destino definido'
 'Industria']


In [205]:
Cat_Permisos

,GID,EXPEDIENTE,FECHA_INI,FECHA_APRO,MES_APRO,ANIO_APRO,PADRON,UNIDAD,DSC_TIPO_T,DSC_DESTIN,DSC_REGIME,DSC_TIPO_O,DSC_CATEGO,CONSTRUCTOR,AREA_EDIF,T_Demora,año_ini,Nuevo_Destino
0,25988748,4113-016778-00,2000-08-10,2000-09-06,9,2000,2,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Propiedad Horizontal,Obra Nueva,Económico o Sencillo,Desconocido,2600,27,2000,Vivienda
1,25988984,2018-4113-98-001306,2018-12-19,2019-03-06,3,2019,8,NaN,OBRAS/CONSTRUCCIONES,Comercio,Común,Regularizacion - Año,Mediano,Desconocido,500,77,2018,Comercio
2,25988984,4113-007575-13,2013-07-03,2013-07-03,7,2013,8,NaN,OBRAS/CONSTRUCCIONES,Comercio,Común,Obra Nueva,Mediano,Desconocido,31500,0,2013,Comercio
3,25989011,4113-016126-98,1998-08-24,1998-12-01,12,1998,9,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Común,Reforma,Mediano,Desconocido,1484,99,1998,Vivienda
4,25987893,4113-014456-00,2000-06-30,2004-03-26,3,2004,15,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Propiedad Horizontal,Modificacion en Obra,Confortable,Desconocido,4985,1365,2000,Vivienda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40781,90215637,GERPC-7-779506,2023-12-08,2024-05-20,5,2024,432190,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Reforma,Desconocido,Desconocido,15100,164,2023,Sin destino definido
40782,90215624,GERPC-7-792529,2023-12-28,2024-09-13,9,2024,432205,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,327365,260,2023,Sin destino definido
40783,90215646,GERPC-7-792051,2023-12-27,2024-05-08,5,2024,432207,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,117230,133,2023,Sin destino definido
40784,93662054,GERPC-7-843816,2024-03-14,2024-08-02,8,2024,432234,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,862714,141,2024,Sin destino definido


In [206]:
Cat_Permisos

,GID,EXPEDIENTE,FECHA_INI,FECHA_APRO,MES_APRO,ANIO_APRO,PADRON,UNIDAD,DSC_TIPO_T,DSC_DESTIN,DSC_REGIME,DSC_TIPO_O,DSC_CATEGO,CONSTRUCTOR,AREA_EDIF,T_Demora,año_ini,Nuevo_Destino
0,25988748,4113-016778-00,2000-08-10,2000-09-06,9,2000,2,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Propiedad Horizontal,Obra Nueva,Económico o Sencillo,Desconocido,2600,27,2000,Vivienda
1,25988984,2018-4113-98-001306,2018-12-19,2019-03-06,3,2019,8,NaN,OBRAS/CONSTRUCCIONES,Comercio,Común,Regularizacion - Año,Mediano,Desconocido,500,77,2018,Comercio
2,25988984,4113-007575-13,2013-07-03,2013-07-03,7,2013,8,NaN,OBRAS/CONSTRUCCIONES,Comercio,Común,Obra Nueva,Mediano,Desconocido,31500,0,2013,Comercio
3,25989011,4113-016126-98,1998-08-24,1998-12-01,12,1998,9,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Común,Reforma,Mediano,Desconocido,1484,99,1998,Vivienda
4,25987893,4113-014456-00,2000-06-30,2004-03-26,3,2004,15,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Propiedad Horizontal,Modificacion en Obra,Confortable,Desconocido,4985,1365,2000,Vivienda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40781,90215637,GERPC-7-779506,2023-12-08,2024-05-20,5,2024,432190,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Reforma,Desconocido,Desconocido,15100,164,2023,Sin destino definido
40782,90215624,GERPC-7-792529,2023-12-28,2024-09-13,9,2024,432205,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,327365,260,2023,Sin destino definido
40783,90215646,GERPC-7-792051,2023-12-27,2024-05-08,5,2024,432207,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,117230,133,2023,Sin destino definido
40784,93662054,GERPC-7-843816,2024-03-14,2024-08-02,8,2024,432234,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,862714,141,2024,Sin destino definido


In [207]:
#Recodificar Tipo de obra
mapeo_DSC_TIPO_O = {
    'Obra Nueva': 'Obra Nueva',
    'Regularizacion - Año' : 'Regularización',
    'Modificacion en Obra' : 'Otros',
    'Reforma': 'Reforma',
    'Reforma a Regularizar': 'Regularización',
    'Incorporacion A.P.H.': 'Incorporacion P.H.',
    'Pilotaje': 'Otros',
    'Estacionamiento': 'Otros',
    'Obra nueva / ampliaci??n': 'Obra Nueva',
    'Demolicion': 'Demolición',
    'A regularizar': 'Regularización',
    'Autorizada': 'Otros',
    'Reciclaje': 'Otros',
    'Regularizaci??n de reforma': 'Regularización',
    'Desconocido' : 'Sin tipo de obra definido',
    'Galpon' : 'Otros',
    'Marquesina':'Otros',
    'Toldo':'Otros',
    'A ocupar':'Otros',
    'Cielo A. Autorizada':'Otros',
    'Cielo A. A regularizar':'Otros',
    '       ':'Sin tipo de obra definido',
    'Incorporaci??n a PH': 'Incorporacion P.H.'         
}

Cat_Permisos['Tipo_Obra'] = Cat_Permisos['DSC_TIPO_O'].map(mapeo_DSC_TIPO_O)

print(Cat_Permisos['Tipo_Obra'].unique())

Cat_Permisos['Cantidad'] = 1

Cat_Permisos


['Obra Nueva' 'Regularización' 'Reforma' 'Otros' 'Incorporacion P.H.'
 'Sin tipo de obra definido' 'Demolición']


,GID,EXPEDIENTE,FECHA_INI,FECHA_APRO,MES_APRO,ANIO_APRO,PADRON,UNIDAD,DSC_TIPO_T,DSC_DESTIN,DSC_REGIME,DSC_TIPO_O,DSC_CATEGO,CONSTRUCTOR,AREA_EDIF,T_Demora,año_ini,Nuevo_Destino,Tipo_Obra,Cantidad
0,25988748,4113-016778-00,2000-08-10,2000-09-06,9,2000,2,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Propiedad Horizontal,Obra Nueva,Económico o Sencillo,Desconocido,2600,27,2000,Vivienda,Obra Nueva,1
1,25988984,2018-4113-98-001306,2018-12-19,2019-03-06,3,2019,8,NaN,OBRAS/CONSTRUCCIONES,Comercio,Común,Regularizacion - Año,Mediano,Desconocido,500,77,2018,Comercio,Regularización,1
2,25988984,4113-007575-13,2013-07-03,2013-07-03,7,2013,8,NaN,OBRAS/CONSTRUCCIONES,Comercio,Común,Obra Nueva,Mediano,Desconocido,31500,0,2013,Comercio,Obra Nueva,1
3,25989011,4113-016126-98,1998-08-24,1998-12-01,12,1998,9,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Común,Reforma,Mediano,Desconocido,1484,99,1998,Vivienda,Reforma,1
4,25987893,4113-014456-00,2000-06-30,2004-03-26,3,2004,15,NaN,OBRAS/CONSTRUCCIONES,Vivienda,Propiedad Horizontal,Modificacion en Obra,Confortable,Desconocido,4985,1365,2000,Vivienda,Otros,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40781,90215637,GERPC-7-779506,2023-12-08,2024-05-20,5,2024,432190,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Reforma,Desconocido,Desconocido,15100,164,2023,Sin destino definido,Reforma,1
40782,90215624,GERPC-7-792529,2023-12-28,2024-09-13,9,2024,432205,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,327365,260,2023,Sin destino definido,Obra Nueva,1
40783,90215646,GERPC-7-792051,2023-12-27,2024-05-08,5,2024,432207,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,117230,133,2023,Sin destino definido,Obra Nueva,1
40784,93662054,GERPC-7-843816,2024-03-14,2024-08-02,8,2024,432234,NaN,OBRAS/CONSTRUCCIONES,Desconocido,Desconocido,Obra nueva / ampliaci??n,Desconocido,Desconocido,862714,141,2024,Sin destino definido,Obra Nueva,1


In [211]:
print(Cat_Permisos.columns)  # Para ver todas las columnas del DataFrame

Index(['GID', 'EXPEDIENTE', 'FECHA_INI', 'FECHA_APRO', 'MES_APRO', 'ANIO_APRO',
       'PADRON', 'UNIDAD', 'DSC_TIPO_T', 'DSC_DESTIN', 'DSC_REGIME',
       'DSC_TIPO_O', 'DSC_CATEGO', 'CONSTRUCTOR', 'AREA_EDIF', 'T_Demora',
       'año_ini', 'Nuevo_Destino', 'Tipo_Obra', 'Cantidad'],
      dtype='object')


In [208]:
# Agrupamos por 'ANIO_APRO', 'MES_APRO', 'Nuevo_Destino', 'Tipo_Obra' y sumamos 'AREA_EDIF' y 'Cantidad'
Tipo_de_obra = Cat_Permisos.groupby(['ANIO_APRO', 'MES_APRO', 'Nuevo_Destino', 'Tipo_Obra'])[['AREA_EDIF', 'Cantidad']].sum().reset_index()

# Renombramos las columnas directamente al hacer el reset_index
Tipo_de_obra.columns = ['ANIO_APRO', 'MES_APRO', 'Nuevo_Destino', 'Tipo_Obra', 'Suma_Area_Edificada', 'Suma_Cantidad']

# Mostramos el DataFrame con los nuevos nombres
print(Tipo_de_obra)


      ANIO_APRO  MES_APRO         Nuevo_Destino                  Tipo_Obra  \
0          1997        10             Industria  Sin tipo de obra definido   
1          1997        10              Vivienda  Sin tipo de obra definido   
2          1997        11              Comercio  Sin tipo de obra definido   
3          1997        11             Industria  Sin tipo de obra definido   
4          1997        11              Vivienda             Regularización   
...         ...       ...                   ...                        ...   
5199       2024        10  Sin destino definido         Incorporacion P.H.   
5200       2024        10  Sin destino definido                 Obra Nueva   
5201       2024        10  Sin destino definido                      Otros   
5202       2024        10  Sin destino definido                    Reforma   
5203       2024        10  Sin destino definido             Regularización   

      Suma_Area_Edificada  Suma_Cantidad  
0                   

In [209]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=Tipo_de_obra, x='Tipo_Obra', y='Suma_Area_Edificada', hue='Nuevo_Destino')

# Añadir título y etiquetas a los ejes
plt.title("Área de Edificación por Tipo de Construcción")
plt.xlabel("Tipo de Construcción")
plt.ylabel("Área Edificada")

# Rotar las etiquetas del eje X
plt.xticks(rotation=45, ha='right')

# Mostrar el gráfico
plt.tight_layout()
plt.show()

C:\Users\vfernand\AppData\Local\Temp\ipykernel_4176\2726111885.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [212]:
resultado

,ANIO_APRO,MES_APRO,Nuevo_Destino,Tipo_Obra,Suma_Area_Edificada,Suma_Cantidad
0,1997,10,Industria,Sin tipo de obra definido,0,1
1,1997,10,Vivienda,Sin tipo de obra definido,0,2
2,1997,11,Comercio,Sin tipo de obra definido,0,3
3,1997,11,Industria,Sin tipo de obra definido,0,2
4,1997,11,Vivienda,Regularización,3663,1
...,...,...,...,...,...,...
5199,2024,10,Sin destino definido,Incorporacion P.H.,57020,3
5200,2024,10,Sin destino definido,Obra Nueva,1769663,13
5201,2024,10,Sin destino definido,Otros,3390698,7
5202,2024,10,Sin destino definido,Reforma,66980,4
